In [1]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from matplotlib.legend_handler import HandlerLine2D
# import json
from pylab import figure
import ast
import os
import pandas as pd
import numpy as np
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
from dtw import dtw
import functools
import math
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
def load_dataframe(filepath):
    data = pd.read_csv(filepath, sep=", ", header=None)
    data[0] = data[0].apply(lambda x: x.replace("[(", ""))
    data[2] = data[2].apply(lambda x: x.replace(")", ""))
    data[3] = data[3].apply(lambda x: x.replace("(", ""))
    data[5] = data[5].apply(lambda x: x.replace(")]", ""))
    for i in range(len(data.columns)):
        data[i] = data[i].apply(lambda x: int(x))
    data.columns = ['accX', 'accY', 'accZ', 'gyrX', 'gyrY', 'gyrZ']
    return data 

In [4]:
load_dataframe('P1-BT/back-1.txt')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,accX,accY,accZ,gyrX,gyrY,gyrZ
0,-15530,-1834,-5055,-28,187,9
1,-15524,-1856,-5060,-1,144,15
2,-15522,-1856,-5070,-48,158,-30
3,-15522,-1856,-5070,-48,158,-30
4,-15522,-1856,-5070,-48,158,-30
5,-15522,-1856,-5070,-48,158,-30
6,-15522,-1856,-5070,-48,158,-30
7,-15532,-1887,-5120,-28,115,-28
8,-15532,-1887,-5120,-28,115,-28
9,-15532,-1887,-5120,-28,115,-28


In [14]:
def create_single_row_df(df):
    df1=df.stack().swaplevel()
    df1.index=df1.index.map('{0[0]}_{0[1]}'.format) 
    df_new = df1.to_frame().T
    c = df_new.shape[1] + 1
    while c <= 2880: # 480 * 6
        df_new[c] = np.nan
        c += 1     
    return df_new

In [22]:
# edit this as per the file name 
labels = {'BFS': 5, 'LRS': 6, 'RLS': 7, 'FBS': 8}

In [23]:
def gesture_overall_dataframe(root_dir, label):
    final_df = pd.DataFrame()
    label_df = pd.DataFrame()
    for fname in os.listdir(root_dir):
        if fname == '.DS_Store':
            continue
        filepath = os.path.join(root_dir, fname)
        df = create_single_row_df(load_dataframe(filepath))
        df['label'] = label
        #dataframes.append(df)
        final_df = final_df.append(pd.DataFrame(df.values.tolist()), ignore_index=True)
    #dataframe = pd.concat([each for each in dataframes],ignore_index=True)    
    return final_df

In [20]:
# p1 = pd.concat([each for each in [p1_back, p1_left, p1_right, p1_front]],ignore_index=True)    

In [21]:
# p1

,0,1,2,3,4,5,6,7,8,9,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
0,-15568.0,-1626.0,-5116.0,-71.0,214.0,7.0,-15570.0,-1642.0,-5151.0,-83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-15582.0,-1563.0,-5154.0,-57.0,188.0,24.0,-15580.0,-1548.0,-5112.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-15564.0,-1876.0,-4929.0,-57.0,200.0,41.0,-15556.0,-1862.0,-4955.0,-61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-15530.0,-1834.0,-5055.0,-28.0,187.0,9.0,-15524.0,-1856.0,-5060.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-15551.0,-1857.0,-5146.0,-36.0,187.0,38.0,-15544.0,-1864.0,-5132.0,-34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-15464.0,-1877.0,-5400.0,41.0,82.0,6.0,-15477.0,-1892.0,-5306.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,-15547.0,-1768.0,-5113.0,-59.0,177.0,21.0,-15548.0,-1738.0,-5133.0,-123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-15552.0,-1541.0,-5167.0,-4.0,157.0,50.0,-15553.0,-1526.0,-5180.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-15514.0,-1816.0,-5203.0,-40.0,165.0,-2.0,-15525.0,-1820.0,-5213.0,-39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-15511.0,-1768.0,-5193.0,-110.0,161.0,15.0,-15504.0,-1770.0,-5179.0,-78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [24]:
def get_dataframe_for_each_participant(participant_number):
    dfs = []
    for l in labels:
        gesture_name = participant_number + "-" + l
        # Temporary adjustment 
#         if gesture_name == 'P2_Left':
#             continue
        dfs.append(gesture_overall_dataframe(gesture_name, labels[l]))
    df =  pd.concat([each for each in dfs],ignore_index=True)      
    return df

In [14]:
get_dataframe_for_each_participant("P1") #P1, P2, P8, 

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2271,2272,2273,2274,2275,2276,2277,2278,2279,2280
0,-13421.0,-1557.0,-10137.0,-44.0,362.0,-20.0,-13438.0,-1592.0,-10134.0,-37.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-13379.0,-1675.0,-10187.0,-25.0,379.0,-36.0,-13367.0,-1662.0,-10121.0,-38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-13362.0,-1695.0,-10161.0,-55.0,426.0,-7.0,-13383.0,-1700.0,-10162.0,-49.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-13144.0,-1888.0,-10396.0,-75.0,424.0,2.0,-13136.0,-1877.0,-10397.0,-77.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-13108.0,-1612.0,-10541.0,-130.0,325.0,1.0,-13108.0,-1612.0,-10541.0,-130.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-13148.0,-2318.0,-10280.0,-38.0,378.0,12.0,-13144.0,-2319.0,-10308.0,-57.0,...,-127.0,381.0,23.0,-13331.0,-2095.0,-10071.0,-127.0,381.0,23.0,1.0
6,-13366.0,-1200.0,-10235.0,15.0,428.0,-17.0,-13366.0,-1200.0,-10235.0,15.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-13430.0,-1341.0,-10081.0,15.0,470.0,-28.0,-13404.0,-1354.0,-10107.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-13382.0,-1596.0,-10150.0,-114.0,334.0,-1.0,-13384.0,-1608.0,-10135.0,-136.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-13390.0,-1665.0,-10154.0,-32.0,423.0,-19.0,-13396.0,-1666.0,-10176.0,-76.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [115]:
def get_dataframe_for_all_participants():
    particiapnts = ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10', 'P11', 'P12']
    dfs = []
    for p in particiapnts:
        dfs.append(get_dataframe_for_each_participant(p))
    df = pd.concat([each for each in dfs],ignore_index=True)      
    return df        

In [116]:
sample_dataframe = get_dataframe_for_all_participants()
sample_dataframe

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,0,1,2,3,4,5,6,7,8,9,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
0,-15568.0,-1626.0,-5116.0,-71.0,214.0,7.0,-15570.0,-1642.0,-5151.0,-83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-15582.0,-1563.0,-5154.0,-57.0,188.0,24.0,-15580.0,-1548.0,-5112.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-15564.0,-1876.0,-4929.0,-57.0,200.0,41.0,-15556.0,-1862.0,-4955.0,-61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-15530.0,-1834.0,-5055.0,-28.0,187.0,9.0,-15524.0,-1856.0,-5060.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-15551.0,-1857.0,-5146.0,-36.0,187.0,38.0,-15544.0,-1864.0,-5132.0,-34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-15464.0,-1877.0,-5400.0,41.0,82.0,6.0,-15477.0,-1892.0,-5306.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,-15547.0,-1768.0,-5113.0,-59.0,177.0,21.0,-15548.0,-1738.0,-5133.0,-123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-15552.0,-1541.0,-5167.0,-4.0,157.0,50.0,-15553.0,-1526.0,-5180.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-15514.0,-1816.0,-5203.0,-40.0,165.0,-2.0,-15525.0,-1820.0,-5213.0,-39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-15511.0,-1768.0,-5193.0,-110.0,161.0,15.0,-15504.0,-1770.0,-5179.0,-78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [125]:
sad = sample_dataframe.copy()
sad

,0,1,2,3,4,5,6,7,8,9,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
0,-15568.0,-1626.0,-5116.0,-71.0,214.0,7.0,-15570.0,-1642.0,-5151.0,-83.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,-15582.0,-1563.0,-5154.0,-57.0,188.0,24.0,-15580.0,-1548.0,-5112.0,24.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,-15564.0,-1876.0,-4929.0,-57.0,200.0,41.0,-15556.0,-1862.0,-4955.0,-61.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,-15530.0,-1834.0,-5055.0,-28.0,187.0,9.0,-15524.0,-1856.0,-5060.0,-1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,-15551.0,-1857.0,-5146.0,-36.0,187.0,38.0,-15544.0,-1864.0,-5132.0,-34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
5,-15464.0,-1877.0,-5400.0,41.0,82.0,6.0,-15477.0,-1892.0,-5306.0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
6,-15547.0,-1768.0,-5113.0,-59.0,177.0,21.0,-15548.0,-1738.0,-5133.0,-123.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
7,-15552.0,-1541.0,-5167.0,-4.0,157.0,50.0,-15553.0,-1526.0,-5180.0,-5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
8,-15514.0,-1816.0,-5203.0,-40.0,165.0,-2.0,-15525.0,-1820.0,-5213.0,-39.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
9,-15511.0,-1768.0,-5193.0,-110.0,161.0,15.0,-15504.0,-1770.0,-5179.0,-78.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [126]:
sad = sad.fillna(727199802211965)
sad

,0,1,2,3,4,5,6,7,8,9,...,2871,2872,2873,2874,2875,2876,2877,2878,2879,2880
0,-15568.0,-1626.0,-5116.0,-71.0,214.0,7.0,-15570.0,-1642.0,-5151.0,-83.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
1,-15582.0,-1563.0,-5154.0,-57.0,188.0,24.0,-15580.0,-1548.0,-5112.0,24.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
2,-15564.0,-1876.0,-4929.0,-57.0,200.0,41.0,-15556.0,-1862.0,-4955.0,-61.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
3,-15530.0,-1834.0,-5055.0,-28.0,187.0,9.0,-15524.0,-1856.0,-5060.0,-1.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
4,-15551.0,-1857.0,-5146.0,-36.0,187.0,38.0,-15544.0,-1864.0,-5132.0,-34.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
5,-15464.0,-1877.0,-5400.0,41.0,82.0,6.0,-15477.0,-1892.0,-5306.0,20.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
6,-15547.0,-1768.0,-5113.0,-59.0,177.0,21.0,-15548.0,-1738.0,-5133.0,-123.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
7,-15552.0,-1541.0,-5167.0,-4.0,157.0,50.0,-15553.0,-1526.0,-5180.0,-5.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
8,-15514.0,-1816.0,-5203.0,-40.0,165.0,-2.0,-15525.0,-1820.0,-5213.0,-39.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0
9,-15511.0,-1768.0,-5193.0,-110.0,161.0,15.0,-15504.0,-1770.0,-5179.0,-78.0,...,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,7.271998e+14,1.0


In [127]:
ddd = pd.DataFrame(sad.iloc[[0],:-1].values)
pd.DataFrame(ddd.values.reshape(480,-1),
                      index=ddd.index.repeat(480))

,0,1,2,3,4,5
0,-1.556800e+04,-1.626000e+03,-5.116000e+03,-7.100000e+01,2.140000e+02,7.000000e+00
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.557000e+04,-1.642000e+03,-5.151000e+03,-8.300000e+01,2.140000e+02,2.800000e+01
0,-1.556800e+04,-1.630000e+03,-5.097000e+03,-1.400000e+01,1.870000e+02,3.000000e+01
0,-1.556800e+04,-1.630000e+03,-5.097000e+03,-1.400000e+01,1.870000e+02,3.000000e+01


In [128]:
def get_dataframe_column_for_comparison(df):
    df = pd.DataFrame(df).T
    dfc = pd.DataFrame(df.values.reshape(480,-1),
                      index=df.index.repeat(480))
    dfc = dfc.replace(727199802211965, np.nan).dropna()
    return dfc.values

In [129]:
def run_dtw(df1, df2):
    dist, cost, acc, path = dtw(get_dataframe_column_for_comparison(df1), get_dataframe_column_for_comparison(df2), dist=lambda x, y: norm(x - y, ord=1))
    return dist

In [130]:
X = sad.iloc[:, :-1].values#sample_dataframe.iloc[:, :-1].values
y = sad.iloc[:, -1].values#sample_dataframe.iloc[:, -1].values

In [131]:
X

array([[-1.55680000e+04, -1.62600000e+03, -5.11600000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14],
       [-1.55820000e+04, -1.56300000e+03, -5.15400000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14],
       [-1.55640000e+04, -1.87600000e+03, -4.92900000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14],
       ...,
       [-1.55000000e+04,  1.60300000e+03, -5.53200000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14],
       [-1.55040000e+04,  1.52900000e+03, -5.52000000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14],
       [-1.54560000e+04,  1.88900000e+03, -5.44800000e+03, ...,
         7.27199802e+14,  7.27199802e+14,  7.27199802e+14]])

In [ ]:
y

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42, stratify=y)

In [134]:
knn = KNeighborsClassifier(n_neighbors=1, metric=run_dtw)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
# train_accuracy[i] = knn.score(X_train, y_train)

In [ ]:
#Compute accuracy on the test set
# test_accuracy[i] = knn.score(X_test, y_test) 

In [618]:
y_pred= knn.predict(X_test)

In [622]:
accuracy_score(y_pred,y_test)

0.8666666666666667

In [627]:
# scores = cross_val_score(knn, X, y, cv=54)

In [628]:
# scores

array([1.  , 0.8 , 1.  , 1.  , 0.5 , 0.75, 1.  , 1.  , 1.  , 1.  , 1.  ,
       0.75, 1.  , 1.  , 1.  , 1.  , 0.5 , 1.  , 1.  , 1.  , 0.75, 1.  ,
       1.  , 0.75, 1.  , 1.  , 0.75, 1.  , 0.75, 1.  , 1.  , 0.75, 1.  ,
       1.  , 0.75, 1.  , 1.  , 0.75, 1.  , 1.  , 1.  , 0.75, 0.75, 0.25,
       1.  , 0.75, 0.75, 1.  , 0.75, 1.  , 1.  , 0.5 , 1.  , 0.75])

In [629]:
# print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.89 (+/- 0.34)
